In [ ]:
import librosa
import librosa.display
import os
import numpy as np
import math
import re 

## Load DataSet Files name

In [ ]:
torgoPath = r"E:\tergo dataset\data and labels" # DataSet path update it 
UASpeech = r"G:\New folder (2)\UASpeech\audio" # DataSet path update it 

In [ ]:
def readDataSetFiles(DataSetPath,labels=True):
    dataSetAllMainFolders= sorted(os.listdir(DataSetPath)) #[f_audio,f_label,m_audio,m_label]
    allDataFilesInDataSetPaths =[DataSetPath + '\\' + Folder+'\\'+File for Folder in dataSetAllMainFolders for File in sorted(os.listdir(DataSetPath + '\\' + Folder)) if re.search('_audio',Folder)] #get all wav files in one list
    if labels:
        allLabelFilesInDataSetPaths=[DataSetPath + '\\' + Folder+'\\'+File for Folder in dataSetAllMainFolders for File in sorted(os.listdir(DataSetPath + '/' + Folder)) if re.search('_label',Folder)] #get all txt files in one list
    else:allLabelFilesInDataSetPaths=[]
    del(dataSetAllMainFolders)
    return allDataFilesInDataSetPaths,allLabelFilesInDataSetPaths


In [ ]:
allDataFilesInDataSetPaths,allLabelFilesInDataSetPaths=readDataSetFiles(torgoPath) #for Torgo Dataset
allDataFilesInDataSetPaths,allLabelFilesInDataSetPaths=readDataSetFiles(UASpeech,labels=False)#for UASpeech Dataset and call readLabelsLinesFromTxt function


In [ ]:
print(len(allDataFilesInDataSetPaths))
print(len(allLabelFilesInDataSetPaths))
print('first data file',allDataFilesInDataSetPaths[:1],'last data file',allDataFilesInDataSetPaths[-1:])
print('first label file',allLabelFilesInDataSetPaths[:1],'last label file',allLabelFilesInDataSetPaths[-1:])

In [ ]:
def readLabelsLinesFromTxt(labelsFiles):
    errorReadLabelTxtFile,allLabelFilesInDataSetPaths=[],[]
    for labelsFileName in labelsFiles:
        try:
            with open(labelsFileName) as labelTxtFile:
                allLabelFilesInDataSetPaths=sorted(labelTxtFile.readlines())
                allLabelFilesInDataSetPaths=[line.split(':')[-1].strip() for line in allLabelFilesInDataSetPaths]
        except:
            errorReadLabelTxtFile.append(labelsFileName)
    return allLabelFilesInDataSetPaths,errorReadLabelTxtFile

In [ ]:
%%time
allLabelFilesInDataSetPaths,errorReadLabelTxtFile=readLabelsLinesFromTxt(UASpeech)

In [ ]:
print(len(allLabelFilesInDataSetPaths))
print(len(errorReadLabelTxtFile))

# set variables as default values 

In [ ]:
SAMPLE_RATE = 16000
AUDIO_DURATION = 10 # measured in seconds


# Extract signal and mfcc feature form waves

In [ ]:
def extractSignal_mfccFeature(allAudioFilesPaths,sampleRate,audioDuration):
    allWavesInDataAsNumpyArray, errorFilesInDataset,mfccFeaturesNumpyArray= [] ,[] ,[] 
    for dataFilePath in allAudioFilesPaths: # all wave files paths
        print(dataFilePath.split('\\')[-1])
        try:
            signal, sampleRate = librosa.load(dataFilePath , sr = sampleRate , duration = audioDuration , res_type='kaiser_fast')#get signals and sampleRate in ane wave (dataFilePath)   
            mfccs = np.mean(librosa.feature.mfcc(y=signal, sr=sampleRate, n_mfcc=40).T,axis=0)  #get mfcc array featture for one wave   
        except :
            errorFilesInDataset.append(dataFilePath) #collect name of file that have error to find it
        feature = np.array(mfccs).reshape([-1,1]) #get mfcc array featture for one wave after reshape it
        mfccFeaturesNumpyArray.append(feature)  #apeend featture array to get list of all waves feature                       
        allWavesInDataAsNumpyArray.append(signal) #apeend signal array to get list of all waves signals
    return mfccFeaturesNumpyArray,allWavesInDataAsNumpyArray,errorFilesInDataset

In [ ]:
%%time
mfccFeaturesAsNumpyArray,allWavesInDataAsNumpyArray,errorFilesInDataset = extractSignal_mfccFeature(allDataFilesInDataSetPaths,SAMPLE_RATE,AUDIO_DURATION)

## Extract Text of audio form txt file

In [ ]:
def extractTextFromTextFiles(LabelOfDataSetPaths):
    allLabelsInDatasetText,errorTextFilePaths=[],[]
    for labelFilePath in LabelOfDataSetPaths:  # all txt files
        try:
            with open(labelFilePath, "r") as labelTextFile: # open label file
                allLabelsInDatasetText.append(labelTextFile.read().strip()) #get labels text in one list without duple spaces or \n
            print(labelFilePath.split('\\')[-1])
        except:
            errorTextFilePaths.append(labelFilePath)
    return allLabelsInDatasetText,errorTextFilePaths

In [ ]:
%%time
allLabelsInDatasetText,errorTextFilePaths=extractTextFromTextFiles(allLabelFilesInDataSetPaths[:10])


In [ ]:
print(allLabelsInDatasetText[:10])
print(len(allLabelsInDatasetText),'\n\n')
print(allWavesInDataAsNumpyArray[0])
print(len(allWavesInDataAsNumpyArray),'\n\n')
print(errorFilesInDataset)

In [ ]:
print(mfccFeaturesAsNumpyArray[0])
print(len(mfccFeaturesAsNumpyArray))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (12,4))
librosa.display.waveplot(allWavesInDataAsNumpyArray[0], sr = SAMPLE_RATE )

In [ ]:
MFCCs = librosa.feature.mfcc(allWavesInDataAsNumpyArray[0], SAMPLE_RATE, n_mfcc=13)
# display MFCCs
plt.figure(figsize=(10,4))
librosa.display.specshow(MFCCs, x_axis = 'time')
plt.colorbar()
plt.title("MFCCs")
plt.tight_layout()
plt.show()

In [ ]:
MFCCs = librosa.feature.mfcc(allWavesInDataAsNumpyArray[0], SAMPLE_RATE, n_mfcc=13)
MFCCs = MFCCs.T
print(MFCCs)

In [ ]:
X = np.array(mfccFeaturesAsNumpyArray)
print(X.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(allLabelsInDatasetText)
classes = list(le.classes_)
print(y.shape)

In [ ]:
from keras.utils import np_utils
y = np_utils.to_categorical(y, num_classes = len(allLabelsInDatasetText))
y = np.array(y)  
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Dropout , Activation , Flatten , LSTM, Input
from keras.optimizers import Adam
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
# Building LSTM model :
model_LSTM = Sequential()
model_LSTM.add(LSTM(units = 50 ,dropout = 0.05 , recurrent_dropout = 0.2 ,input_shape =(x_train.shape[1:])))
model_LSTM.add(Dense(485,activation = 'softmax'))
model_LSTM.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
model_LSTM.summary()

In [ ]:
model_LSTM.fit(x_train,y_train,epochs = 100 ,batch_size = 120)
score = model_LSTM.evaluate(x_test,y_test)